Named Entity Recognition (NER)

In [1]:
import spacy

# Load the pre-trained NLP model from spacy
nlp = spacy.load('en_core_web_sm')

# The sentence for which we want to perform NER
sentence = "Barack Obama was the 44th President of the United States."

# Process the sentence using the NLP model
doc = nlp(sentence)

# Print the named entities recognized in the sentence
print("Named Entities in the sentence:")
for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

Named Entities in the sentence:
Barack Obama: PERSON
44th: ORDINAL
the United States: GPE


Language Model

In [3]:
import nltk
from collections import defaultdict, Counter
import random

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')

# Sample text data (corpus)
corpus = [
    "I love natural language processing",
    "I love machine learning",
    "I enjoy learning new things",
    "Natural language processing is fascinating"
]

# Tokenize the sentences into words
tokenized_corpus = [nltk.word_tokenize(sentence.lower()) for sentence in corpus]

# Create bigrams from the tokenized corpus
bigrams = []
for sentence in tokenized_corpus:
    bigrams.extend(list(nltk.bigrams(sentence)))

# Calculate bigram frequencies
bigram_freq = defaultdict(Counter)
for w1, w2 in bigrams:
    bigram_freq[w1][w2] += 1

# Calculate bigram probabilities
bigram_prob = defaultdict(dict)
for w1 in bigram_freq:
    total_count = float(sum(bigram_freq[w1].values()))
    for w2 in bigram_freq[w1]:
        bigram_prob[w1][w2] = bigram_freq[w1][w2] / total_count

# Function to generate text using the bigram model
def generate_sentence(start_word, num_words=10):
    current_word = start_word
    sentence = [current_word]
    for _ in range(num_words - 1):
        if current_word in bigram_prob:
            next_word = random.choices(
                list(bigram_prob[current_word].keys()),
                list(bigram_prob[current_word].values())
            )[0]
            sentence.append(next_word)
            current_word = next_word
        else:
            break
    return ' '.join(sentence)

# Generate a sentence starting with the word "i"
generated_sentence = generate_sentence("i", num_words=4)
print("Generated sentence:", generated_sentence)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...


Generated sentence: i enjoy learning new


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Transformers

In [4]:
import torch
import torch.nn as nn
import math

class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, ffn_dim, max_seq_len, num_classes):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Embedding(input_dim, model_dim)
        self.model_dim = model_dim
        self.layers = nn.ModuleList([
            TransformerLayer(model_dim, num_heads, ffn_dim)
            for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        seq_len = x.size(1)
        positional_encoding = self._generate_positional_encoding(seq_len)
        x = self.embedding(x) + positional_encoding
        for layer in self.layers:
            x = layer(x)
        return self.fc_out(x.mean(dim=1))

    def _generate_positional_encoding(self, seq_len):
        positional_encoding = torch.zeros(seq_len, self.model_dim)
        for pos in range(seq_len):
            for i in range(0, self.model_dim, 2):
                positional_encoding[pos, i] = math.sin(pos / (10000 ** (i / self.model_dim)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((i + 1) / self.model_dim)))
        return positional_encoding.unsqueeze(0)

class TransformerLayer(nn.Module):
    def __init__(self, model_dim, num_heads, ffn_dim):
        super(TransformerLayer, self).__init__()
        self.multihead_attention = nn.MultiheadAttention(embed_dim=model_dim, num_heads=num_heads)
        self.norm1 = nn.LayerNorm(model_dim)
        self.ffn = nn.Sequential(
            nn.Linear(model_dim, ffn_dim),
            nn.ReLU(),
            nn.Linear(ffn_dim, model_dim)
        )
        self.norm2 = nn.LayerNorm(model_dim)

    def forward(self, x):
        attn_output, _ = self.multihead_attention(x, x, x)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        return x

# Example usage:
input_dim = 10000  # Vocabulary size
model_dim = 512
num_heads = 8
num_layers = 6
ffn_dim = 2048
max_seq_len = 100
num_classes = 10

# Instantiate the model
model = TransformerModel(input_dim, model_dim, num_heads, num_layers, ffn_dim, max_seq_len, num_classes)

# Example input (batch_size=32, sequence_length=50)
x = torch.randint(0, input_dim, (32, 50))

# Forward pass
output = model(x)
print("Output shape:", output.shape)

Output shape: torch.Size([32, 10])
